In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!ls "drive/MyDrive/Colab Notebooks/multidoc_summarization"

demo_unsupervised_text_summarization.ipynb  ViMs.zip
KTLAB-200document-clusters.zip


In [3]:
!unzip "drive/MyDrive/Colab Notebooks/multidoc_summarization/KTLAB-200document-clusters.zip"
!unzip "drive/MyDrive/Colab Notebooks/multidoc_summarization/ViMs.zip"

Streaming output truncated to the last 5000 lines.
  inflating: ViMs/summary/Cluster_296/.DS_Store  
  inflating: __MACOSX/ViMs/summary/Cluster_296/._.DS_Store  
  inflating: ViMs/summary/Cluster_296/0.gold.txt  
  inflating: __MACOSX/ViMs/summary/Cluster_296/._0.gold.txt  
  inflating: ViMs/summary/Cluster_296/1.gold.txt  
  inflating: __MACOSX/ViMs/summary/Cluster_296/._1.gold.txt  
  inflating: ViMs/summary/Cluster_050/.DS_Store  
  inflating: __MACOSX/ViMs/summary/Cluster_050/._.DS_Store  
  inflating: ViMs/summary/Cluster_050/0.gold.txt  
  inflating: __MACOSX/ViMs/summary/Cluster_050/._0.gold.txt  
  inflating: ViMs/summary/Cluster_050/1.gold.txt  
  inflating: __MACOSX/ViMs/summary/Cluster_050/._1.gold.txt  
  inflating: ViMs/summary/Cluster_262/.DS_Store  
  inflating: __MACOSX/ViMs/summary/Cluster_262/._.DS_Store  
  inflating: ViMs/summary/Cluster_262/0.gold.txt  
  inflating: __MACOSX/ViMs/summary/Cluster_262/._0.gold.txt  
  inflating: ViMs/summary/Cluster_262/1.gold.txt  


In [4]:
!pip install sentence-transformers
!pip install evaluate
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=4a01a3252b7f55d64407deaaf96c21719b5ad3bee6d9af63e2e405952961950a
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
!pip install rouge-metric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 KB 3.4 MB/s eta 0:00:00


In [18]:
from sklearn.cluster import KMeans
import os
import evaluate
from glob import glob
rouge = evaluate.load('rouge')
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
def concat_doc(files):
  docs = []
  for file in files:
    with open(file, "r") as f:
      doc = f.read()
      docs.append(doc)
  return " ".join(docs).replace("\xa0", " ")

In [7]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
model = SentenceTransformer("keepitreal/vietnamese-sbert")

In [8]:
from tqdm import tqdm

In [9]:
def generate_summary(source, k, selected_num):
  source_sents = sent_tokenize(source)
  source_vec = model.encode(source_sents, batch_size = 16, show_progress_bar  = False)
  clusterer = KMeans(n_clusters = k, n_init  = 'auto')
  clusterer.fit(source_vec)
  centroids = clusterer.cluster_centers_
  dists = cdist(centroids, source_vec)
  selected_by_cent = dists.argsort(axis = 1)[:, :selected_num]
  selected_sents = selected_by_cent.reshape(-1,)

  summ = []
  for ind in selected_sents:
    summ.append(source_sents[ind])
  sum_pred = " ".join(summ)
  return sum_pred

# Experiment with Vietnamese-MDS

In [15]:
sum_preds = []
sums = []
k = 1
selected_num = 8
from scipy.spatial.distance import cdist
all_dirs = glob("clusters/cluster_*")
with tqdm(total = len(all_dirs)) as pbar:
  for dir in all_dirs:
    body_files = glob(f"{dir}/*.body.txt")
    sum_files = [f"{dir}/{dir.split('/')[-1]}.ref1.txt", f"{dir}/{dir.split('/')[-1]}.ref2.txt"]
    source = concat_doc(body_files)
    sum = concat_doc(sum_files)
    sum_pred = generate_summary(source, k, selected_num)
    sum_preds.append(sum_pred)
    sums.append(sum)
    pbar.update(1)

results = rouge.compute(predictions=sum_preds,
                         references=sums)
print(results)

100%|██████████| 199/199 [00:42<00:00,  4.65it/s]


{'rouge1': 0.7809072019690575, 'rouge2': 0.5519243744840079, 'rougeL': 0.4367888708272929, 'rougeLsum': 0.436822799338892}


# Experiment with ViMs

In [13]:
from scipy.spatial.distance import cdist
configs = [(7, 2)]
for config in configs:
  sum_preds = []
  sums = []
  k = config[0]
  selected_num = config[1]
  all_sum_dirs = glob("ViMs/summary/*")
  with tqdm(total = len(all_sum_dirs)) as pbar:
    for dir in all_sum_dirs:
      cluster_name = dir.split("/")[-1]

      body_files = glob(f"ViMs/original/{cluster_name}/original/*.txt")
      sum_files = glob(f"{dir}/*.gold.txt")
      source = concat_doc(body_files)
      sum = concat_doc(sum_files)
      sum_pred = generate_summary(source, k, selected_num)
      sum_preds.append(sum_pred)
      sums.append(sum)
      pbar.update(1)
  results = rouge.compute(predictions=sum_preds,
                          references=sums)
  print(f"k: {k}, num_sent: {selected_num}")
  print(results)

100%|██████████| 300/300 [02:18<00:00,  2.16it/s]


k: 7, num_sent: 2
{'rouge1': 0.7134947232178358, 'rouge2': 0.5084123380813506, 'rougeL': 0.36671183118630635, 'rougeLsum': 0.6798212016028289}
